## RecSys

#### Загрузка датасета, ETL, EDA

In [3]:
import numpy as np
import pandas as pd

In [2]:
links = pd.read_csv('C:/Users/rossi/Desktop/MLOps/Books/Dataset/links.csv', sep = ',')
movies = pd.read_csv('C:/Users/rossi/Desktop/MLOps/Books/Dataset/movies.csv', sep = ',')
tags = pd.read_csv('C:/Users/rossi/Desktop/MLOps/Books/Dataset/tags.csv', sep = ',')
ratings = pd.read_csv('C:/Users/rossi/Desktop/MLOps/Books/Dataset/ratings.csv', sep = ',')

In [3]:
tags.head()

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200


In [4]:
tags = tags.drop(['timestamp'], axis=1) # удаляем ненужную фичу timestamp

In [5]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [6]:
movies['genres'].value_counts() # смотрим распределение классов жанров

Drama                                                  1053
Comedy                                                  946
Comedy|Drama                                            435
Comedy|Romance                                          363
Drama|Romance                                           349
                                                       ... 
Action|Crime|Horror|Mystery|Thriller                      1
Adventure|Animation|Children|Comedy|Musical|Romance       1
Action|Adventure|Animation|Comedy|Crime|Mystery           1
Children|Comedy|Fantasy|Sci-Fi                            1
Action|Animation|Comedy|Fantasy                           1
Name: genres, Length: 951, dtype: int64

In [7]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [8]:
ratings = ratings.drop(['timestamp'], axis=1) # удаляем ненужную фичу timestamp

In [9]:
print(tags.shape)
print(movies.shape)
print(ratings.shape)

(3683, 3)
(9742, 3)
(100836, 3)


#### 1. Простая RecSys, выводящая топ фильмов по рейтингу

In [30]:
rating_count = pd.DataFrame(ratings, columns=['movieId','rating']) # формируем датасет из требуемых данных

In [11]:
rating_count.sort_values('rating', ascending=False).drop_duplicates().head(10) #сортируем топ фильмов по убыванию

,movieId,rating
34031,3147,5.0
50717,1080,5.0
18166,487,5.0
18165,480,5.0
76157,3308,5.0
18158,296,5.0
18157,260,5.0
50696,608,5.0
50698,750,5.0
50699,858,5.0


In [12]:
rating_count.groupby(by = 'rating')['movieId'].max() # группируем фильмы (выводим по 1 фильму в каждой группе оценок рейтинга)

rating
0.5    184641
1.0    190221
1.5    190215
2.0    185033
2.5    189713
3.0    193567
3.5    193587
4.0    193609
4.5    191005
5.0    187717
Name: movieId, dtype: int64

In [13]:
# формируем датасет из 10 любых фильмов в каждой группе

most_rated_books = pd.DataFrame([184641, 190221, 190215, 185033, 189713, 193567, 193587, 193609, 191005, 187717], index=np.arange(10), columns=['movieId']) 
most_rated_books
detail = pd.merge(most_rated_books, movies, on='movieId')
detail

,movieId,title,genres
0,184641,Fullmetal Alchemist 2018 (2017),Action|Adventure|Fantasy
1,190221,Hommage à Zgougou (et salut à Sabine Mamou) (2...,Documentary
2,190215,Liquid Truth (2017),Drama
3,185033,I Kill Giants (2018),Drama|Fantasy|Thriller
4,189713,BlacKkKlansman (2018),Comedy|Crime|Drama
5,193567,anohana: The Flower We Saw That Day - The Movi...,Animation|Drama
6,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation
7,193609,Andrew Dice Clay: Dice Rules (1991),Comedy
8,191005,Gintama (2017),Action|Adventure|Comedy|Sci-Fi
9,187717,Won't You Be My Neighbor? (2018),Documentary


In [14]:
# итоговая таблица с подбором фильма в каждой группе оценок

detail2 = pd.merge(detail, rating_count, on='movieId')
detail2

,movieId,title,genres,rating
0,184641,Fullmetal Alchemist 2018 (2017),Action|Adventure|Fantasy,0.5
1,190221,Hommage à Zgougou (et salut à Sabine Mamou) (2...,Documentary,1.0
2,190215,Liquid Truth (2017),Drama,1.5
3,185033,I Kill Giants (2018),Drama|Fantasy|Thriller,2.0
4,189713,BlacKkKlansman (2018),Comedy|Crime|Drama,2.5
5,193567,anohana: The Flower We Saw That Day - The Movi...,Animation|Drama,3.0
6,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation,3.5
7,193609,Andrew Dice Clay: Dice Rules (1991),Comedy,4.0
8,191005,Gintama (2017),Action|Adventure|Comedy|Sci-Fi,4.5
9,187717,Won't You Be My Neighbor? (2018),Documentary,5.0


In [68]:
# выводим топ фильмов по рейтингу в каждом жанре

a = pd.merge(ratings, movies, on='movieId')
a.sort_values(by=['rating', 'genres'], ascending=False).drop_duplicates(subset=['title']).head(10)

,userId,movieId,rating,title,genres
23409,57,2921,5.0,High Plains Drifter (1973),Western
31693,240,383,5.0,Wyatt Earp (1994),Western
51374,29,82459,5.0,True Grit (2010),Western
71307,66,3037,5.0,Little Big Man (1970),Western
76105,62,128360,5.0,The Hateful Eight (2015),Western
82609,414,2401,5.0,Pale Rider (1985),Western
82637,66,3074,5.0,Jeremiah Johnson (1972),Western
93295,572,2670,5.0,Run Silent Run Deep (1958),War
2998,1,457,5.0,"Fugitive, The (1993)",Thriller
19278,325,190,5.0,Safe (1995),Thriller


In [64]:
a[a['genres'] == 'Action']['genres'].head(5) # топ-5 фильмов в жанре Action

18826    Action
29524    Action
29525    Action
29526    Action
29527    Action
Name: genres, dtype: object

#### Рекомендашка на основе кореляции Пирсона (collaborative-base)

In [16]:
ratings['movieId'].value_counts() # смотрим кол-во оценок по каждлму фильму

356       329
318       317
296       307
593       279
2571      278
         ... 
86279       1
86922       1
5962        1
87660       1
163981      1
Name: movieId, Length: 9724, dtype: int64

In [21]:
rat = pd.DataFrame(ratings.groupby('movieId')['rating'].mean()) # посчитаем среднюю оценку по каждому фильму
rat.head()

,rating
movieId,
1,3.920930
2,3.431818
3,3.259615
4,2.357143
5,3.071429


In [22]:
rat.sort_values('rating', ascending=False).head() # выводим топ фильмов по рейтингу

,rating
movieId,
88448,5.0
100556,5.0
143031,5.0
143511,5.0
143559,5.0


In [23]:
user_rating= pd.pivot_table(data=ratings, values='rating', index='userId', columns='movieId') # Подготовим таблицу данных для анализа
user_rating.head()

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,NaN,4.0,NaN,NaN,4.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
correlation_matrix  = user_rating.corr(method='pearson') # Рассчитаем корреляционную матрицу по всему набору книг 
correlation_matrix.head(10)

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
movieId,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.330978,0.487109,1.000000,0.310971,0.106465,0.208402,0.968246,0.095913,-0.021409,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.330978,1.000000,0.419564,NaN,0.562791,0.163510,0.430261,0.415227,0.277350,0.016626,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.487109,0.419564,1.000000,NaN,0.602266,0.345069,0.554088,0.333333,0.458591,-0.050276,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1.000000,NaN,NaN,1.000000,0.654654,NaN,0.203653,NaN,NaN,0.870388,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,0.310971,0.562791,0.602266,0.654654,1.000000,0.291302,0.609119,0.555556,0.319173,0.218263,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,0.106465,0.163510,0.345069,NaN,0.291302,1.000000,-0.123897,-0.801784,-0.172891,0.420222,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,0.208402,0.430261,0.554088,0.203653,0.609119,-0.123897,1.000000,0.816497,0.745356,0.186891,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,0.968246,0.415227,0.333333,NaN,0.555556,-0.801784,0.816497,1.000000,NaN,0.557086,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,0.095913,0.277350,0.458591,NaN,0.319173,-0.172891,0.745356,NaN,1.000000,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
# отберем пользователей, которые оценили конкретную книгу № 10, убедимся, что рейтинг не равен нулю
OneManOut_rating = user_rating[10]
OneManOut_rating[OneManOut_rating>=0]

userId
6      3.0
8      2.0
11     3.0
19     2.0
21     5.0
      ... 
597    3.0
599    3.5
602    3.0
608    4.0
609    4.0
Name: 10, Length: 132, dtype: float64

In [26]:
# оценим сходство с использованием корреляции Пирсона

similar_to_OneManOut = user_rating.corrwith(OneManOut_rating)

corr_OneManOut = pd.DataFrame(similar_to_OneManOut, columns=['PearsonR'])
corr_OneManOut.dropna(inplace=True)
corr_OneManOut.head()

C:\Users\rossi\anaconda3\envs\dzi\lib\site-packages\numpy\lib\function_base.py:2845: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar, dtype=dtype)
C:\Users\rossi\anaconda3\envs\dzi\lib\site-packages\numpy\lib\function_base.py:2704: RuntimeWarning: divide by zero encountered in divide
  c *= np.true_divide(1, fact)


,PearsonR
movieId,
1,-0.021409
2,0.016626
3,-0.050276
4,0.870388
5,0.218263


In [29]:
OneManOut_corr_summary = corr_OneManOut.join(rat)
# выведем самые похожие книги, отсортировав их при равном сходстве по рейтингу книги
most_rated_books2=OneManOut_corr_summary.sort_values(by = ['PearsonR', 'rating'], ascending=False).head(10)
detail3 = pd.merge(most_rated_books2, movies, on='movieId')
detail3

,movieId,PearsonR,rating,title,genres
0,98154,1.0,4.500000,Lincoln (2012),Drama|War
1,2360,1.0,4.458333,"Celebration, The (Festen) (1998)",Drama
2,7842,1.0,4.375000,Dune (2000),Drama|Fantasy|Sci-Fi
3,176,1.0,4.307692,Living in Oblivion (1995),Comedy
4,26606,1.0,4.250000,"Chinese Ghost Story, A (Sinnui yauwan) (1987)",Action|Fantasy|Horror|Romance
5,84414,1.0,4.250000,All-Star Superman (2011),Action|Adventure|Animation|Sci-Fi
6,2563,1.0,4.166667,Dangerous Beauty (1998),Drama
7,4160,1.0,4.125000,"Widow of St. Pierre, The (Veuve de Saint-Pierr...",Drama|Romance
8,104069,1.0,4.125000,Louis C.K.: Oh My God (2013),Comedy
9,47999,1.0,4.071429,Jesus Camp (2006),Documentary|Drama


#### 3. RecSys на основе содержимого (contend-base, матрица TF-IDF и косинусная мера подобия)

In [69]:
from sklearn.metrics.pairwise import linear_kernel
from sklearn.feature_extraction.text import TfidfVectorizer

In [75]:
movies = pd.read_csv('C:/Users/rossi/Desktop/MLOps/Books/Dataset/movies.csv', sep = ',')

In [89]:
movies.head(5)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [84]:
movies.shape

(9742, 3)

In [90]:
movies['genres'] = movies['genres'].str.replace('|', ' ') # заменяем в столбце genre знак на пробел
movies.head(5)

C:\Users\rossi\AppData\Local\Temp\ipykernel_11920\1774091024.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  movies['genres'] = movies['genres'].str.replace('|', ' ')


,movieId,title,genres
0,1,Toy Story (1995),Adventure Animation Children Comedy Fantasy
1,2,Jumanji (1995),Adventure Children Fantasy
2,3,Grumpier Old Men (1995),Comedy Romance
3,4,Waiting to Exhale (1995),Comedy Drama Romance
4,5,Father of the Bride Part II (1995),Comedy


In [96]:
movies[movies.genres == '(no genres listed)'].value_counts() # смотрим, у какого кол-ва фильмов отсутстует жанр

Series([], dtype: int64)

In [94]:
movies.drop(movies[movies.genres == '(no genres listed)'].index, inplace=True) # удаляем строки, где отсутствует жанр

In [95]:
movies.shape

(9708, 3)

In [97]:
# Удаляем из текста стоп-слова (частицы, предлоги, местоимения, дефисы, кавычки и т.д.)
movies_tfidf = TfidfVectorizer(stop_words='english') #если опция - russian, то можно русские слова
# replace NaN with empty strings
#movies_description['genre'] = movies_description['genre'].fillna('') #заменяем пробелы на пустые ячейки
# Рассчитываем матрицу TF-IDF, требуемую для расчета косинусной меры подобия текстов
movies_1 = movies_tfidf.fit_transform(movies['genres'])

In [98]:
movies_1.shape # выведенный результат означает, что в базе для описания 9708 книг используется 21 слово. 

(9708, 21)

In [101]:
# рассчитаем косинусную меру подобия с использованием линейного ядра
cosine_similarity = linear_kernel(movies_1, movies_1)

In [127]:
# Получим попарные оценки сходства всех фильмов по сравнению с фильмом, переданным по индексу, отсортировав их и получив топ-5

# здесь 2 - это индекс книги в наборе данных
similarity_scores = list(enumerate(cosine_similarity[2]))
similarity_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)
similarity_scores = similarity_scores[1:6]

# Получим индекс похожих книг
movies_index = [i[0] for i in similarity_scores]

# Выведем топ-5 наиболее похожих книг, используя индексацию на основе целочисленного местоположения (iloc)
print (movies['title'].iloc[movies_index])

6                          Sabrina (1995)
35                        Clueless (1995)
57                   Two if by Sea (1996)
60     French Twist (Gazon maudit) (1995)
103                   If Lucy Fell (1996)
Name: title, dtype: object


In [128]:
# проверим, корректно ли подобрались фильмы по жанру к фильму с индексом 2

movies[movies['title'].isin(['Grumpier Old Men (1995)', 'Sabrina (1995)', 'Clueless (1995)', 'Two if by Sea (1996)',
                             'French Twist (Gazon maudit) (1995)', 'If Lucy Fell (1996)'])]

,movieId,title,genres
2,3,Grumpier Old Men (1995),Comedy Romance
6,7,Sabrina (1995),Comedy Romance
35,39,Clueless (1995),Comedy Romance
57,64,Two if by Sea (1996),Comedy Romance
60,68,French Twist (Gazon maudit) (1995),Comedy Romance
103,118,If Lucy Fell (1996),Comedy Romance


#### 4. RecSys с использованием "Bag of Words"

In [4]:
kino = pd.read_csv('C:/Users/rossi/Desktop/MLOps/Books/Dataset/kinopoisk-top250.csv')

In [135]:
kino.head()

,rating,movie,year,country,rating_ball,overview,director,screenwriter,actors,url_logo
0,0,Побег из Шоушенка,1994,США,9.111,Бухгалтер Энди Дюфрейн обвинён в убийстве собс...,Фрэнк Дарабонт,Фрэнк Дарабонт; Стивен Кинг,Тим Роббинс; Морган Фриман; Боб Гантон; Уильям...,'https://st.kp.yandex.net/images/film_iphone/i...
1,1,Зеленая миля,1999,США,9.062,Пол Эджкомб — начальник блока смертников в тюр...,Фрэнк Дарабонт,Фрэнк Дарабонт; Стивен Кинг,Том Хэнкс; Дэвид Морс; Бонни Хант; Майкл Кларк...,'https://st.kp.yandex.net/images/film_iphone/i...
2,2,Форрест Гамп,1994,США,8.913,От лица главного героя Форреста Гампа; слабоум...,Роберт Земекис,Эрик Рот; Уинстон Грум,Том Хэнкс; Робин Райт; Салли Филд; Гэри Синиз;...,'https://st.kp.yandex.net/images/film_iphone/i...
3,3,Список Шиндлера,1993,США,8.817,Фильм рассказывает реальную историю загадочног...,Стивен Спилберг,Стивен Зеллиан; Томас Кенилли,Лиам Нисон; Бен Кингсли; Рэйф Файнс; Кэролайн ...,'https://st.kp.yandex.net/images/film_iphone/i...
4,4,1+1,2011,Франция,8.807,Пострадав в результате несчастного случая; бог...,Оливье Накаш; Эрик Толедано,Оливье Накаш; Филипп Поццо ди Борго; Эрик Т...,Франсуа Клюзе; Омар Си; Анн Ле Ни; Одри Флеро;...,'https://st.kp.yandex.net/images/film_iphone/i...


In [139]:
kino.info() # смотрим тип данных, видим, что столбец "Описание" имеет тип Object

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   rating        250 non-null    int64  
 1   movie         250 non-null    object 
 2   year          250 non-null    int64  
 3   country       250 non-null    object 
 4   rating_ball   250 non-null    float64
 5   overview      250 non-null    object 
 6   director      250 non-null    object 
 7   screenwriter  250 non-null    object 
 8   actors        250 non-null    object 
 9   url_logo      250 non-null    object 
dtypes: float64(1), int64(2), object(7)
memory usage: 19.7+ KB


In [141]:
kino.overview = kino.overview.astype(str) # возможно данный пункт и не нужен
kino.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   rating        250 non-null    int64  
 1   movie         250 non-null    object 
 2   year          250 non-null    int64  
 3   country       250 non-null    object 
 4   rating_ball   250 non-null    float64
 5   overview      250 non-null    object 
 6   director      250 non-null    object 
 7   screenwriter  250 non-null    object 
 8   actors        250 non-null    object 
 9   url_logo      250 non-null    object 
dtypes: float64(1), int64(2), object(7)
memory usage: 19.7+ KB


In [136]:
kino.isnull().sum() # проверяем на пустые ячейки в столбце описания

rating          0
movie           0
year            0
country         0
rating_ball     0
overview        0
director        0
screenwriter    0
actors          0
url_logo        0
dtype: int64

In [148]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\rossi\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [152]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\rossi\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [154]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\rossi\AppData\Roaming\nltk_data...


True

In [5]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re

In [6]:
STOPWORDS = set(stopwords.words('russian')) #описание не меньше 4 и не больше 200 слов
MIN_WORDS = 4
MAX_WORDS = 200

#паттерны для удаления спецюсимволов и всяких шумов
PATTERN_S = re.compile("\'s")  # matches `'s` from text  
PATTERN_RN = re.compile("\\r\\n") #matches `\r` and `\n`
PATTERN_PUNC = re.compile(r"[^\w\s]") # matches all non 0-9 A-z whitespace
PATTERN_HZ = re.compile("\\xa0")


def clean_text(text):
    """
    Series of cleaning. String to lower case, remove non words characters and numbers (punctuation, curly brackets etc).
        text (str): input text
    return (str): modified initial text
    """
    text = str(text).lower()  # lowercase text
    # replace the matched string with ' '
    text = re.sub(PATTERN_S, ' ', text)
    text = re.sub(PATTERN_RN, ' ', text)
    text = re.sub(PATTERN_PUNC, ' ', text)
    text = re.sub(PATTERN_HZ, ' ', text)                    
    return text

# выделение только значимых слов в тексте
def tokenizer(sentence, min_words=MIN_WORDS, max_words=MAX_WORDS, stopwords=STOPWORDS, lemmatize=True):
    """
    Lemmatize, tokenize, crop and remove stop words.
    Args:
      sentence (str)
      min_words (int)
      max_words (int)
      stopwords (set of string)
      lemmatize (boolean)
    returns:
      list of string
    """
    if lemmatize:
        stemmer = WordNetLemmatizer()
        tokens = [stemmer.lemmatize(w) for w in word_tokenize(sentence)]
    else:
        tokens = [w for w in word_tokenize(sentence)]
    token = [w for w in tokens if (len(w) > min_words and len(w) < max_words
                                                        and w not in stopwords)]
    return tokens    

def clean_sentences(df):
    """
    Remove irrelavant characters (in new column clean_sentence).
    Lemmatize, tokenize words into list of words (in new column tok_lem_sentence).
    Args: 
      df (dataframe)
     returns:
      df
    """
    print('Cleaning sentences...')
    kino['clean_sentence'] = kino['overview'].apply(clean_text)
    kino['tok_lem_sentence'] = kino['overview'].apply(
        lambda x: tokenizer(x, min_words=MIN_WORDS, max_words=MAX_WORDS, stopwords=STOPWORDS))
    return kino
    
kino = clean_sentences(kino)

Cleaning sentences...


In [7]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def extract_best_indices(m, topk, mask=None):
    """
    Use sum of the cosine distance over all tokens ans return best mathes.
    m (np.array): cos matrix of shape (nb_in_tokens, nb_dict_tokens)
    topk (int): number of indices to return (from high to lowest in order)
    """
    # return the sum on all tokens of cosinus for each sentence
    if len(m.shape) > 1:
        cos_sim = np.mean(m, axis=0) 
    else: 
        cos_sim = m
    index = np.argsort(cos_sim)[::-1] # from highest idx to smallest score 
    if mask is not None:
        assert mask.shape == m.shape
        mask = mask[index]
    else:
        mask = np.ones(len(cos_sim))
    mask = np.logical_or(cos_sim[index] != 0, mask) #eliminate 0 cosine distance
    best_index = index[mask][:topk]  
    return best_index

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np


def get_recommendations_tfidf(overview, tfidf_mat):
    
    """
    Return the database sentences in order of highest cosine similarity relatively to each 
    token of the target sentence. 
    """
    # Embed the query sentence
    tokens_query = [str(tok) for tok in tokenizer(overview)]
    embed_query = vectorizer.transform(tokens_query)
    # Create list with similarity between query and dataset
    mat = cosine_similarity(embed_query, tfidf_mat)
    # Best cosine distance for each token independantly
    best_index = extract_best_indices(mat, topk=3)
    return best_index

# Adapt stop words
token_stop = tokenizer(' '.join(stopwords.words('russian')), lemmatize=False)

# Fit TFIDF
vectorizer = TfidfVectorizer(stop_words=token_stop, tokenizer=tokenizer) 
tfidf_mat = vectorizer.fit_transform(kino['overview'].values) # -> (num_sentences, num_vocabulary)

# Return best threee matches between query and dataset
test_sentence = 'вторая мировая война' 
best_index = get_recommendations_tfidf(test_sentence, tfidf_mat)

display(kino[['movie', 'overview']].iloc[best_index])

,movie,overview
73,Летят журавли,Рассказ о людях; в чьи судьбы безжалостно втор...
85,Дневник памяти,Это история отношений юноши и девушки из разны...
82,Судьба человека,Фильм рассказывает о русском солдате; которого...


In [10]:
kino.loc[73, 'overview'] # проверяем подобранный фильм на тему "Вторая мировая война" по описанию

'Рассказ о\xa0людях; в\xa0чьи судьбы безжалостно вторглась война. Не\xa0все смогли с\xa0честью вынести это\xa0испытание. В\xa0центре киноповести\xa0—\xa0трагическая история двух влюбленных; которых разлучила война.'

In [11]:
kino.loc[82, 'clean_sentence'] # # проверяем подобранный фильм на тему "Вторая мировая война" по описанию

'фильм рассказывает о русском солдате  которого война подвергла страшным испытаниям  лишила дома и семьи  бросила в концлагерь  но судьба не сломила его дух   он выжил  отстоял своё право быть человеком  сохранил способность любить '

In [188]:
pip install gensim

     ---------------------------------------- 24.0/24.0 MB 2.8 MB/s eta 0:00:00
     ---------------------------------------- 56.8/56.8 kB 1.5 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [23]:
from gensim.models.word2vec import Word2Vec

def is_word_in_model(word, model):
    """
    Check on individual words ``word`` that it exists in ``model``.
    """
    assert type(model).__name__ == 'KeyedVectors'
    is_in_vocab = word in model.key_to_index.keys()
    return is_in_vocab

def predict_w2v(query_sentence, dataset, model, topk=3):
    query_sentence = query_sentence.split()
    in_vocab_list, best_index = [], [0]*topk
    for w in query_sentence:
        # remove unseen words from query sentence
        if is_word_in_model(w, model.wv):
            in_vocab_list.append(w)
    # Retrieve the similarity between two words as a distance
    if len(in_vocab_list) > 0:
        sim_mat = np.zeros(len(dataset))  # TO DO
        for i, data_sentence in enumerate(dataset):
            if data_sentence:
                sim_sentence = model.wv.n_similarity(
                        in_vocab_list, data_sentence)
            else:
                sim_sentence = 0
            sim_mat[i] = np.array(sim_sentence)
        # Take the five highest norm
        best_index = np.argsort(sim_mat)[::-1][:topk]
    return best_index

# Create model
word2vec_model = Word2Vec(min_count=0, workers = 8, vector_size=300) 
# Prepare vocab
word2vec_model.build_vocab(kino.tok_lem_sentence.values)
# Train
word2vec_model.train(kino.tok_lem_sentence.values, total_examples=word2vec_model.corpus_count, epochs=30)

test_sentence1 = 'зарождается любовь' 
# Predict
best_index1 = predict_w2v(test_sentence1, kino['tok_lem_sentence'].values, word2vec_model)    
display(kino[['movie', 'overview']].iloc[best_index1])

,movie,overview
117,Артист,Голливуд; 1927 год. Звезда немого кино Джордж ...
84,Назад в будущее 2,Продолжение фантастической истории о приключен...
131,Афера,В 1930-е годы двое отпетых мошенников пытаются...


In [27]:
kino.loc[131, 'clean_sentence'] # # проверяем подобранный фильм на тему "Вторая мировая война" по описанию

'в 1930 е годы двое отпетых мошенников пытаются отомстить главарю мафии за смерть их общего друга с помощью одной из самых неожиданных афер в истории кино  которая к тому же заканчивается удивительным и забавным образом '